# Merge Together the Stock Info

In [56]:
import os 
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder


In [57]:
all_data = []


for filename in os.listdir("data"):
    if filename != "films_stocks_clean.csv": 
        file_path = os.path.join("data", filename)
        df = pd.read_csv(file_path)
        df["Stock"] = filename[:-4]
        all_data.append(df)

merged_df = pd.concat(all_data, ignore_index=True)
merged_df["Date"] = pd.to_datetime(merged_df["Date"])
merged_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock
0,2012-03-30,7.50,8.24,7.06,7.34,7.34,5048600.0,ENPH
1,2012-04-02,7.65,8.20,7.40,7.81,7.81,495500.0,ENPH
2,2012-04-03,7.75,7.80,7.50,7.64,7.64,316600.0,ENPH
3,2012-04-04,7.77,7.77,7.31,7.45,7.45,165100.0,ENPH
4,2012-04-05,7.48,7.55,7.37,7.43,7.43,104300.0,ENPH


# Make movie data mergable

In [58]:

df = pd.read_csv("movie_data/movies_with_details.csv")

df['Released'] = pd.to_datetime(df['Released'], errors='coerce', dayfirst=False)
df['Date'] = df['Released'].dt.strftime('%Y-%m-%d')
df = df.drop(columns=['Released'])



# Merge Stock and Movie Data

In [59]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')

df = df[(df['Date'] >= '1999-01-01') & (df['Date'] <= '2019-12-31')]
merged_df = merged_df[(merged_df['Date'] >= '1999-01-01') & (merged_df['Date'] <= '2019-12-31')]

final_df = pd.merge(merged_df, df, on='Date', how='outer')  # or use 'outer', 'left', etc.
df = final_df.dropna()
df = df.pivot_table(
    index=[col for col in df.columns if col != 'Stock'],  # Keep all columns except 'Stock'
    columns='Stock',
    aggfunc='size',  # Count occurrences of each stock
    fill_value=0  # Fill with 0 where there is no occurrence
).reset_index()
df = df.drop(columns = ["Date", "Title", "Director", "Writer", "Actors", "Awards", "Description", "Poster URL"])
df['Box Office'] = df['Box Office'].replace({r'[\$,]': ''}, regex=True).astype(float)
df['IMDB Votes'] = df['IMDB Votes'].replace({r'[,]': ''}, regex=True).astype(float)
df['Runtime'] = df['Runtime'].str.replace(' min', '', regex=False).astype(float)


df = df.rename(columns={'^SPX': 'SPX'})
df = df.dropna()

columns_to_encode = ['Genre', 'Rated', 'Country', 'Language']
encoder = LabelEncoder()
for column in columns_to_encode:
    df[column] = encoder.fit_transform(df[column])

In [60]:
df.to_csv("merged_data/merged.csv")